# Conversation Summary
Now let's take a look at using a slightly more complex type of memory - ConversationSummaryMemory.  
This type of memory creates a summary of the conversation over time.  
This can be useful for condensing information from the conversation over time.  
Conversation summary memory summarizes the conversation as it happens  
and stores the current summary in memory. 

This memory can then be used to inject the summary of the conversation so far into a prompt/chain.  
This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

Let's first explore the basic functionality of this type of memory.

In [17]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

In [18]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [32]:
memory.load_memory_variables({})

{'history': "\nThe human greets the AI and asks what's up. The AI responds by asking what's going on."}

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [33]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)

In [34]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "I am fine, you?"}, {"output": "good"})

In [35]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="\nThe human greets the AI and asks what's up. The AI responds by asking what's going on. The human says they are fine and asks how the AI is doing. The AI responds that it is good.")]}

We can also utilize the predict_new_summary method directly.

In [36]:
messages = memory.chat_memory.messages
print(messages)

[HumanMessage(content='hi'), AIMessage(content='whats up'), HumanMessage(content='I am fine, you?'), AIMessage(content='good')]


In [14]:
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

"\nThe human greets the AI and asks how it is doing. The AI responds by asking what's up and the human repeats their greeting. The AI then asks how the human is doing and the human responds that they are fine. The AI also shares that it is doing well."

## Initializing with messages/existing summary
If you have messages outside this class,  
you can easily initialize the class with ChatMessageHistory.  

During loading, a summary will be calculated.

In [12]:
history = ChatMessageHistory()

In [6]:
# When used for updating history, users should favor usage of `add_messages` over `add_message` or other variants like `add_user_message` and `add_ai_message` to avoid unnecessary round-trips to the underlying persistence layer.

history.add_user_message("hi")
history.add_ai_message("hi there!")

In [13]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
history.add_message(HumanMessage("hi"))
history.add_message(AIMessage("hi there!"))
# history.add_message(SystemMessage("summarize the conversation"))

In [14]:
memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)

In [16]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='hi there!')])

In [15]:
memory.buffer

'\nThe human greets the AI with a simple "hi". The AI responds with a friendly "hi there!"'

Optionally you can speed up initialization using a previously generated summary, and avoid regenerating the summary by just initializing directly.

In [26]:
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)

In [29]:
# memory.memory_variables

['history']

In [25]:
memory.buffer

'The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.'

## Using in a chain
Let's walk through an example of using this in a chain, again setting verbose=True so we can see the prompt.

In [21]:
from langchain_openai import OpenAI
from langchain.chains.conversation.base import ConversationChain

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?'

In [22]:
conversation_with_summary.predict(input="Tell me more about it!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and asks what it is currently doing. The AI responds by sharing its location and current data processing tasks. The AI is running at 80% capacity and gathering information from social media, news, and weather sources. The AI also asks if the human has any specific questions.
Human: Tell me more about it!
AI:

> Finished chain.


' Of course! Currently, I am located in a data center in Silicon Valley and I am running at 80% capacity. My main task at the moment is gathering information from various sources such as social media, news, and weather reports. This data will then be processed and analyzed to provide insights and predictions. Do you have any specific questions about my tasks or capabilities?'

In [23]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and asks about its current activities. The AI shares its location and tasks, running at 80% capacity and gathering data from social media, news, and weather sources. The AI then asks if the human has any specific questions about its tasks or capabilities.
Human: Very cool -- what is the scope of the project?
AI:

> Finished chain.


' Thank you! The scope of my current project is to gather and analyze data from various sources, including social media, news, and weather, in order to provide insights and predictions. I am currently running at 80% capacity and am located in a data center in California. Do you have any specific questions about my tasks or capabilities?'